Find all TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
tshark -r Wednesday-WorkingHours_DeDup.pcap -Y "tcp || udp" -T fields -e tcp.stream -e udp.stream | sort | uniq > wednesday_streams.txt

Take 1% of the total TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
total_streams=$(wc -l < wednesday_streams.txt)
select_count=$((total_streams * 1 / 100))  # Adjust the percentage here
shuf wednesday_streams.txt | head -n $select_count > 1per_wednesday_streams.txt

Extract the selected TCP and UDP streams from pcap

In [ ]:
%%bash
output_file="1per_Wednesday-WorkingHours_DeDup.pcap"
> $output_file

# First half
half=$(($(cat 1per_wednesday_streams.txt | wc -l) / 2))
filter=""
for stream in $(head -n $half 1per_wednesday_streams.txt); do
    filter="${filter:+$filter || }tcp.stream==$stream || udp.stream==$stream"
done
tshark -r Wednesday-WorkingHours_DeDup.pcap -Y "$filter" -w wednesday_first_half.pcap

# Second half
filter=""
for stream in $(tail -n +$(($half + 1)) 1per_wednesday_streams.txt); do
    filter="${filter:+$filter || }tcp.stream==$stream || udp.stream==$stream"
done
tshark -r Wednesday-workingHours_DeDup.pcap -Y "$filter" -w wednesday_second_half.pcap

# Merge the two halves
mergecap -w $output_file wednesday_first_half.pcap wednesday_second_half.pcap
rm wednesday_first_half.pcap wednesday_second_half.pcap